In [1]:
from lxml import etree
import pandas as pd

In [2]:
# Generate track dictionaries
xml_path = '/Users/kevinmarlis/Developer/Music Analytics/Library.xml'
tracks = []
track_dicts = []

for event, element in etree.iterparse(xml_path):
    if element.text == 'Tracks':
        results = element.getnext()

for element in results:
    if element.tag == 'dict':
        tracks.append(element)

for item in tracks:
    track = {}
    for element in item:
        if element.tag == 'key':
            track[element.text] = element.getnext().text
    track_dicts.append(track)

In [46]:
df = pd.DataFrame(track_dicts)
cols_to_drop = ['Kind', 'Size', 'Disc Number', 'Disc Count',
       'Track Number', 'Date Modified', 'Bit Rate',
       'Sample Rate', 'Artwork Count', 'Track Type', 'Location', 'File Folder Count', 
       'Library Folder Count', 'Normalization', 'Volume Adjustment', 
       'Play Date', 'Apple Music', 'Movement Name', 'Grouping',
       'Movement Number', 'Movement Count', 'Part Of Gapless Album', 'Comments', 'Album Loved', 'Matched', 'BPM',
       'Playlist Only', 'Explicit', 'Sort Composer', 'Clean', 'Protected',
       'File Type', 'Track Count']
df = df.drop(cols_to_drop, axis=1)
df.head()

,Track ID,Name,Artist,Album Artist,Album,Work,Genre,Total Time,Year,Date Added,...,Sort Album,Sort Album Artist,Sort Artist,Persistent ID,Skip Count,Skip Date,Sort Name,Composer,Release Date,Compilation
0,3000,Everything's Ok [Pretty Mary K](David McConnel...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,278700,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,4E0E450EECD74EDA,NaN,NaN,NaN,NaN,NaN,NaN
1,3003,A Distorted Reality Is Now a Necessity to Be F...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,190824,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,85EEC4BE78C8DEC4,1,2020-08-15T17:56:36Z,Distorted Reality Is Now a Necessity to Be Fre...,NaN,NaN,NaN
2,3006,Suicide Machine (Fritz Machaud New Monkey Stud...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,182543,2003,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,748C7375168B1E23,1,2020-06-26T20:41:39Z,NaN,NaN,NaN,NaN
3,3009,Coast to Coast [Instrumental](David McConnell ...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,302132,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,D1CC527ECAAE3EA0,NaN,NaN,NaN,NaN,NaN,NaN
4,3012,Don't Go Down [Acoustic Version](Jon Brion Two...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,215232,2001,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,A3CFFE0C010A0BA1,NaN,NaN,Don't Go Down,NaN,NaN,NaN


In [47]:
df[df['Date Added'] > '2020']

,Track ID,Name,Artist,Album Artist,Album,Work,Genre,Total Time,Year,Date Added,...,Sort Album,Sort Album Artist,Sort Artist,Persistent ID,Skip Count,Skip Date,Sort Name,Composer,Release Date,Compilation
0,3000,Everything's Ok [Pretty Mary K](David McConnel...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,278700,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,4E0E450EECD74EDA,NaN,NaN,NaN,NaN,NaN,NaN
1,3003,A Distorted Reality Is Now a Necessity to Be F...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,190824,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,85EEC4BE78C8DEC4,1,2020-08-15T17:56:36Z,Distorted Reality Is Now a Necessity to Be Fre...,NaN,NaN,NaN
2,3006,Suicide Machine (Fritz Machaud New Monkey Stud...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,182543,2003,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,748C7375168B1E23,1,2020-06-26T20:41:39Z,NaN,NaN,NaN,NaN
3,3009,Coast to Coast [Instrumental](David McConnell ...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,302132,2002,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,D1CC527ECAAE3EA0,NaN,NaN,NaN,NaN,NaN,NaN
4,3012,Don't Go Down [Acoustic Version](Jon Brion Two...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,215232,2001,2020-06-26T16:04:32Z,...,Unreleased,Elliott Smith,Elliott Smith,A3CFFE0C010A0BA1,NaN,NaN,Don't Go Down,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15734,71678,As the Night Starts Closing In,Experimental Audio Research,Experimental Audio Research,Phenomena 256,NaN,Electronic,627461,1996,2020-12-21T22:36:54Z,...,Phenomena 256,NaN,Experimental Audio Research,FCFE0BE247234335,NaN,NaN,As the Night Starts Closing In,Experimental Audio Research,1996-06-02T12:00:00Z,NaN
15735,71681,Ring Modulator,Experimental Audio Research,Experimental Audio Research,Phenomena 256,NaN,Electronic,171389,1996,2020-12-21T22:36:54Z,...,Phenomena 256,NaN,Experimental Audio Research,52B697659099A46D,NaN,NaN,Ring Modulator,Experimental Audio Research,1996-06-02T12:00:00Z,NaN
15736,71684,Sub Aqua (Left Channel) Tidal (Centre Channel)...,Experimental Audio Research,Experimental Audio Research,Phenomena 256,NaN,Electronic,589767,1996,2020-12-21T22:36:54Z,...,Phenomena 256,NaN,Experimental Audio Research,CF28CC35CE715E6E,NaN,NaN,Sub Aqua (Left Channel) Tidal (Centre Channel)...,Experimental Audio Research,1996-06-02T12:00:00Z,NaN
15737,71687,Space Themes Pt. 1 & 2 (Tribute to John Cage i...,Experimental Audio Research,Experimental Audio Research,Phenomena 256,NaN,Electronic,481933,1996,2020-12-21T22:36:54Z,...,Phenomena 256,NaN,Experimental Audio Research,AAE3EE67C5E4B197,NaN,NaN,Space Themes Pt. 1 & 2 (Tribute to John Cage i...,Experimental Audio Research,1996-06-02T12:00:00Z,NaN
